# TSTR GM Dataset F

In [2]:
#import libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import os
print('Libraries imported!!')

Libraries imported!!


In [3]:
#define directory of functions and actual directory
HOME_PATH = '' #home path of the project
FUNCTIONS_DIR = "EVALUATION FUNCTIONS/UTILITY"
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(HOME_PATH + FUNCTIONS_DIR)
#import functions for data labelling analisys
from utility_evaluation import DataPreProcessor
from utility_evaluation import train_evaluate_model

#change directory to actual directory
os.chdir(ACTUAL_DIR)
print('Functions imported!!')

Functions imported!!


## 1. Read data

In [4]:
#read real dataset
train_data = pd.read_csv(HOME_PATH + 'SYNTHETIC DATASETS/GM/F_IndianLiverPatient_Synthetic_GM.csv')
categorical_columns = ['gender','class']
for col in categorical_columns :
    train_data[col] = train_data[col].astype('category')
train_data

,age,gender,TB,DB,alkphos,sgpt,sgot,TP,ALB,A_G,class
0,68,Male,0.792221,0.210326,403.231073,41.466674,33.224376,5.532269,2.299157,0.739703,1.0
1,62,Female,1.081347,0.299965,432.280680,29.344795,45.448756,6.762787,3.424076,1.132054,1.0
2,66,Male,1.106183,0.228158,409.387072,182.941301,74.747186,7.259657,4.059811,0.881810,1.0
3,18,Male,0.319369,0.211002,123.339810,33.420530,39.358963,8.525534,4.605769,1.557012,2.0
4,28,Male,0.689095,0.791855,41.390511,37.560963,49.239559,7.501857,3.313626,0.706537,2.0
...,...,...,...,...,...,...,...,...,...,...,...
461,44,Male,0.555285,0.120601,200.579550,18.119024,20.245458,7.024714,2.973412,0.638638,1.0
462,45,Male,0.441402,0.276043,176.890784,40.251383,37.730839,5.729123,3.276946,1.346909,2.0
463,23,Male,1.294539,0.289846,224.054337,38.982170,123.888386,7.224159,3.751692,0.944799,2.0
464,56,Male,0.516323,0.512852,307.627940,30.908007,44.850122,5.357072,2.377225,0.284098,1.0


In [6]:
#read test data
test_data = pd.read_csv(HOME_PATH + 'REAL DATASETS/TEST DATASETS/F_IndianLiverPatient_Real_Test.csv')
for col in categorical_columns :
    test_data[col] = test_data[col].astype('category')
test_data

,age,gender,TB,DB,alkphos,sgpt,sgot,TP,ALB,A_G,class
0,19,Male,1.4,0.8,178.0,13.0,26.0,8.0,4.6,1.30,2.0
1,12,Male,1.0,0.2,719.0,157.0,108.0,7.2,3.7,1.00,1.0
2,60,Male,5.7,2.8,214.0,412.0,850.0,7.3,3.2,0.78,1.0
3,42,Female,0.5,0.1,162.0,155.0,108.0,8.1,4.0,0.90,1.0
4,40,Male,14.5,6.4,358.0,50.0,75.0,5.7,2.1,0.50,1.0
...,...,...,...,...,...,...,...,...,...,...,...
112,60,Male,0.9,0.3,168.0,16.0,24.0,6.7,3.0,0.80,1.0
113,54,Female,1.4,0.7,195.0,36.0,16.0,7.9,3.7,0.90,2.0
114,47,Female,3.0,1.5,292.0,64.0,67.0,5.6,1.8,0.47,1.0
115,27,Male,1.3,0.6,106.0,25.0,54.0,8.5,4.8,-100000.00,2.0


In [7]:
target = 'class'
#quick look at the breakdown of class values
print('Train data')
print(train_data.shape)
print(train_data.groupby(target).size())
print('#####################################')
print('Test data')
print(test_data.shape)
print(test_data.groupby(target).size())

Train data
(466, 11)
class
1.0    331
2.0    135
dtype: int64
#####################################
Test data
(117, 11)
class
1.0    87
2.0    30
dtype: int64


## 2. Pre-process training data

In [8]:
target = 'class'
categorical_columns = ['gender']
numerical_columns = train_data.select_dtypes(include=['int64','float64']).columns.tolist()
categories = [np.array([0, 1])]

data_preprocessor = DataPreProcessor(categorical_columns, numerical_columns, categories)
x_train = data_preprocessor.preprocess_train_data(train_data.loc[:, train_data.columns != target])
y_train = train_data.loc[:, target]

x_train.shape, y_train.shape

((466, 11), (466,))

## 3. Preprocess test data

In [9]:
x_test = data_preprocessor.preprocess_test_data(test_data.loc[:, test_data.columns != target])
y_test = test_data.loc[:, target]
x_test.shape, y_test.shape

((117, 11), (117,))

## 4. Create a dataset to save the results

In [10]:
results = pd.DataFrame(columns = ['model','accuracy','precision','recall','f1'])
results

,model,accuracy,precision,recall,f1


## 4. Train and evaluate Random Forest Classifier

In [11]:
rf_results = train_evaluate_model('RF', x_train, y_train, x_test, y_test)
results = results.append(rf_results, ignore_index=True)
rf_results

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished


,model,accuracy,precision,recall,f1
0,RF,0.7094,0.6547,0.7094,0.6688


## 5. Train and Evaluate KNeighbors Classifier

In [12]:
knn_results = train_evaluate_model('KNN', x_train, y_train, x_test, y_test)
results = results.append(knn_results, ignore_index=True)
knn_results

,model,accuracy,precision,recall,f1
0,KNN,0.7436,0.6843,0.7436,0.6492


## 6. Train and evaluate Decision Tree Classifier

In [13]:
dt_results = train_evaluate_model('DT', x_train, y_train, x_test, y_test)
results = results.append(dt_results, ignore_index=True)
dt_results

,model,accuracy,precision,recall,f1
0,DT,0.6068,0.6152,0.6068,0.6109


## 7. Train and evaluate Support Vector Machines Classifier

In [14]:
svm_results = train_evaluate_model('SVM', x_train, y_train, x_test, y_test)
results = results.append(svm_results, ignore_index=True)
svm_results

[LibSVM]

,model,accuracy,precision,recall,f1
0,SVM,0.4188,0.7856,0.4188,0.3934


## 8. Train and evaluate Multilayer Perceptron Classifier

In [15]:
mlp_results = train_evaluate_model('MLP', x_train, y_train, x_test, y_test)
results = results.append(mlp_results, ignore_index=True)
mlp_results

Iteration 1, loss = 0.67301539
Iteration 2, loss = 0.62231830
Iteration 3, loss = 0.59497764
Iteration 4, loss = 0.58310826
Iteration 5, loss = 0.57567845
Iteration 6, loss = 0.57025363
Iteration 7, loss = 0.56495291
Iteration 8, loss = 0.55997856
Iteration 9, loss = 0.55712722
Iteration 10, loss = 0.55410486
Iteration 11, loss = 0.55142137
Iteration 12, loss = 0.54805287
Iteration 13, loss = 0.54519334
Iteration 14, loss = 0.54231025
Iteration 15, loss = 0.53912763
Iteration 16, loss = 0.53570333
Iteration 17, loss = 0.53246164
Iteration 18, loss = 0.52987108
Iteration 19, loss = 0.52744147
Iteration 20, loss = 0.52428225
Iteration 21, loss = 0.52100802
Iteration 22, loss = 0.51751555
Iteration 23, loss = 0.51522428
Iteration 24, loss = 0.51098224
Iteration 25, loss = 0.50945954
Iteration 26, loss = 0.50843622
Iteration 27, loss = 0.50410100
Iteration 28, loss = 0.50007731
Iteration 29, loss = 0.49615051
Iteration 30, loss = 0.49254250
Iteration 31, loss = 0.48923807
Iteration 32, los

,model,accuracy,precision,recall,f1
0,MLP,0.7094,0.663,0.7094,0.676


## 9. Save results file

In [16]:
results.to_csv('RESULTS/models_results_gm.csv', index=False)
results

,model,accuracy,precision,recall,f1
0,RF,0.7094,0.6547,0.7094,0.6688
1,KNN,0.7436,0.6843,0.7436,0.6492
2,DT,0.6068,0.6152,0.6068,0.6109
3,SVM,0.4188,0.7856,0.4188,0.3934
4,MLP,0.7094,0.6630,0.7094,0.6760
